# (Разведочный анализ данных) Exploratory data analisys


- AREA_TYPE : тип района, где расположен объект недвижимости.
- AVAILABILITY : доступна ли недвижимость в настоящее время или нет.
- LOCATION : Место, где находится недвижимость.
- SIZE : Количество спален
- SOCIETY : Тип района, в котором находится собственность.
- TOTAL_SQFT : общая площадь объекта в квадратных футах.
- BATH : Количество ванных комнат
- BALCONY : Количество балконов
- PRICE : Стоимость недвижимости в лакхах¶

В качестве задания вам предлагается повторить данные действия для набора данных
- [Somerville Happiness Survey Data Set](http://archive.ics.uci.edu/ml/datasets/Somerville+Happiness+Survey)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.preprocessing import StandardScaler

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/yakushinav/omo/main/data/Bengaluru_House_Data.csv')
print(df.head())

In [ ]:
df.columns

In [ ]:
len(df.columns)

In [ ]:
df.info()

# Смотрим на каждый признак в отдельности

## Что нас интересует?
- Тип (числовая / категориальная)
- Смысл
- Влияние на целевую переменную

В итоге анализа делаем __вывод__.

## Целевая переменная 

In [ ]:
df['price'].describe()

In [ ]:
sns.distplot(df['price']);

In [ ]:
print("Skewness: %f" % df['price'].skew())
print("Kurtosis: %f" % df['price'].kurt())

- Распределение отличается от нормального
- Коэффициент асимметрии > 0
- Коэффициент эксцесса (островершинности)  — мера остроты пика распределения случайной величины


## 1 Анализ пропущенных значений

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull())

In [ ]:
(df.isnull().sum()/len(df))*100

## Категориальные переменные

In [ ]:
#40% данных в поле society пропущено. Удаляем
df.drop('society',axis=1,inplace=True)

In [ ]:
#Заменяем пропущенные значения на моду
df.location.replace(to_replace=np.NaN,value=df.location.mode()[0],inplace=True)
df.area_type.replace(to_replace=np.NaN,value=df.area_type.mode()[0],inplace=True)

In [ ]:
#Поле size содержит данные такого вида 2 BHK. Выделим число отдельно
df['size']=df['size'].str.split(' ',expand=True)[0].astype(np.number)
df.bath=df.bath.astype(np.number)

In [ ]:
#Заменим на бинарный признак доступно/недоступно
df.availability=(df.availability=='Ready To Move')
df.availability.replace({True:1,False:0},inplace=True)
df.availability=df.availability.astype(object)
df.availability.head()

In [ ]:
df.isnull().sum() 

## Числовые переменные

In [ ]:
#Заменяем пропуски в данных на конкретное значение
df.bath=df.bath.astype(np.number)
df['bath'] = df['bath'].fillna(1)
df.balcony = df.balcony.astype(np.number)
df['balcony'] = df['balcony'].fillna(0)
df.size=df.size.astype(np.number)
df['size'] = df['size'].fillna(1)


In [ ]:
#Заменяем пропуски в данных на основе интерполяции
df['total_sqft'] = df['total_sqft'].interpolate(method='polynomial', order=2)
df['price'] = df['price'].interpolate(method='polynomial', order=2)


In [ ]:
sns.heatmap(df.isnull())

In [ ]:
(df.isnull().sum()/len(df))*100

## 2 Удаление выбросов

In [ ]:
df.boxplot()

In [ ]:
# Удалим выбросы, которые выходят далеко за 1 и 3 квартили
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
IQR = q3-q1
IQR

df = df[~((df) > (q3 + (1.5 * IQR))).any(axis=1)]

df.head()

In [ ]:
df.boxplot()

In [ ]:
df.shape

## 3. Разведочный анализ

In [ ]:
sns.violinplot(y = df.price)

In [ ]:
# Распределение недвижимости по доступности и локации
plt.bar(x=df[df['availability'] == 1].groupby('location')['availability'].agg('count').sort_values(ascending=False).head(10).index,height=df[df['availability'] == 1].groupby('location')['availability'].agg('count').sort_values(ascending=False).head(10))
plt.xticks(rotation=45)
plt.ylabel('Доступность')

In [ ]:
# Распределение данных по типу недвижимости
sns.countplot(df.area_type)
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# Распределение по доступности
plt.figure(figsize=(8,5))
sns.countplot(df.availability)
plt.show()

In [ ]:
# Зависимость цены от площади
plt.figure(figsize=(15,5))
sns.scatterplot(x="total_sqft", y="price", data=df)
plt.show()

In [ ]:
# Распределение по количеству спален
df10 = df.copy()
df10['size'] = np.round(df10['size'])
plt.bar(x=df10.groupby('size')['price'].agg('mean').sort_values(ascending=False).head(10).index,height=df10.groupby('size')['price'].agg('mean').sort_values(ascending=False).head(10))
plt.ylabel('Цена')
plt.xlabel('Количество спален')

# Самые дорогие районы в Бангалоре

In [ ]:
plt.plot(df.groupby(by='location')['price'].agg('mean').sort_values(ascending=False).head(10))
plt.xlabel('Район')
plt.ylabel('Цена')
plt.xticks(rotation=45)

# Самые дорогие районы в Бангалоре по отношению цена/площадь

In [ ]:
df['per_sqft'] = df['price']/df['total_sqft']

In [ ]:
plt.plot(df.groupby(by='location')['per_sqft'].agg('mean').sort_values(ascending=False).head(10))
plt.xticks(rotation = 45)
plt.xlabel('Район')
plt.ylabel('Average Price / Sq.ft in Lakhs')

In [ ]:
df = df.drop('per_sqft',axis=1)

In [ ]:
# Тепловая карта
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(),annot=True,cbar=False)
plt.show()

## Стандартизация

In [ ]:
df.head()

In [ ]:
# Выдлим категориальные и числовые признаки
data_cat=df.select_dtypes(exclude=np.number)
data_num=df.select_dtypes(np.number)
data_cat=data_cat.reset_index()

In [ ]:
# Стандартизация это перевод к виду Среднее значение = 0, стандартное отклонение = 1
ss=StandardScaler()
data_num_scaled=pd.DataFrame(ss.fit_transform(data_num),columns=data_num.columns)
data_num_scaled=data_num_scaled.reset_index()
data_num_scaled.head()

In [ ]:
# Соединяем данные
df=pd.concat([data_num_scaled,data_cat],axis=1).drop('index',axis=1)
df.head()

# Далее заменяем категориальные данные на числовые коды

In [ ]:
# Заменим категориальные данные на числовые коды из словаря
encoder = LabelEncoder()
df['location'] = encoder.fit_transform(df['location'])
df['location'] = df['location'].astype(np.number)


In [ ]:
# Заменим категориальные данные на бинарные поля
df=pd.get_dummies(df,columns=['area_type'])
df.head()

In [ ]:
print(df.head())